In [ ]:
from langchain_openai.llms import OpenAI
model = OpenAI()
completion = model.invoke('Hi there!') 
# Hi!
completions = model.batch(['Hi there!', 'Bye!'])
# ['Hi!', 'See you!']
for token in model.stream('Bye!'):
    print(token)
    # Good
    # bye
    # !

In [ ]:
# Imperative Composition
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
# the building blocks
template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}'),
])
model = ChatOpenAI()
# combine them in a function
# @chain decorator adds the same Runnable interface for any function you write
@chain
def chatbot(values):
    prompt = template.invoke(values)
    return model.invoke(prompt)
# use it
result = chatbot.invoke({
    "question": "Which model providers offer LLMs?"
})

#Output:
#AIMessage(content="Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library offer LLMs.")

In [ ]:
# stream way for the output
@chain
def chatbot(values):
    prompt = template.invoke(values)
    for token in model.invoke(prompt):
        yield token
for part in chatbot.stream({
    "question": "Which model providers offer LLMs?"
}):
    print(part)

#Output:
#AIMessageChunk(content="Hugging")
#AIMessageChunk(content=" Face's
#AIMessageChunk(content=" `transformers`
#...

In [ ]:
# for asynchronous execution
@chain
async def chatbot(values):
    prompt = await template.ainvoke(values)
    return await model.ainvoke(prompt)
result = await chatbot.ainvoke({
    "question": "Which model providers offer LLMs?"
})

# Output:
# > AIMessage(content="Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library offer LLMs.")


In [ ]:
# Declarative Composition
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
# the building blocks
template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}'),
])
model = ChatOpenAI()
# combine them with the | operator
chatbot = template | model
# use it
result = chatbot.invoke({
    "question": "Which model providers offer LLMs?"
})

#Output:
#AIMessage(content="Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library offer LLMs.")

In [ ]:
#Crucially, the last line is the same between the two examples—that is, you use the function and the LCEL sequence in the same way, with invoke/stream/batch.
chatbot = template | model
for part in chatbot.stream({
    "question": "Which model providers offer LLMs?"
}):
    print(part)
    # > AIMessageChunk(content="Hugging")
    # > AIMessageChunk(content=" Face's
    # > AIMessageChunk(content=" `transformers`
    # ...

In [ ]:
chatbot = template | model
result = await chatbot.ainvoke({
    "question": "Which model providers offer LLMs?"
})